# Lista 8 z WAS: Testowanie hipotez 

In [1]:
import numpy as np
import pandas as pd

## Zadanie 1

In [2]:
n = 47
corr_full = pd.DataFrame(np.array([
    [1, 0.6, -0.36, -0.53],
    [0.6 ,1, -0.46, -0.37],
    [-0.36, -0.46, 1, 0.37],
    [-0.53, -0.37, 0.37, 1]
]))

corr_full.columns = list(range(1,5))
corr_full.index= list(range(1,5))
corr_full

,1,2,3,4
1,1.00,0.60,-0.36,-0.53
2,0.60,1.00,-0.46,-0.37
3,-0.36,-0.46,1.00,0.37
4,-0.53,-0.37,0.37,1.00


### Zadanie 1a: testowanie korelacji cząstkowej 

W podpunkcie a. najpierw wyznaczamy współczynnik $r := \rho_{12|3}$ korzystając tym razem ze wzoru  z zadania 1 z listy 6:
$$\rho_{ij|k} = \frac{\rho_{ij} - \rho_{ik}\rho_{jk}}{\sqrt{(1 - \rho^2_{ik})(1 - \rho_{jk}^2)}}.$$

In [3]:
def get_partial_ij_k(corr: pd.DataFrame, i:int, j:int, k:int) -> float:
    ij = corr.loc[i,j]
    ik = corr.loc[i,k]
    jk = corr.loc[j,k]
    return (ij - ik*jk)/np.sqrt((1 - ik**2)*(1 - jk**2))

r = get_partial_ij_k(corr_full, 1, 2, 3)
print(f"Wspolczynnik korelacji 12|3 wyniosl: {r:.3f}")

Wspolczynnik korelacji 12|3 wyniosl: 0.524


Dla porównania niżej wyliczamy ten sam współczynnik z użyciem funkcji z listy 7 (sposób macierzowy):

In [4]:
import utils_7 as L7
L7.get_partial_ij_ks(corr_full,1,2,[3]).round(3)

0.524

W dalszej częsci podpunktu a. testujemy hipotezę $H_0: r = 0$ przeciwko $H_1: r \neq 0$ z użyciem testu dwustronnego ze statystyką testową
$$T = \hat r \sqrt{\frac{N - q - 2}{1 - r^2}},$$
gdzie $N$ jest liczebnością próby, a $q$ liczbą zmiennych warunkujących. Statystyka ta ma rozkład t-Studenta z $N-2-q$ stopniami swobody (patrz str. 7 wykładu 7). Odrzucimy hipotezę zerową, jeśli wartość bezwzględna T będzie większa niż kwantyl $1 - \frac{\alpha}{2}.$ 

In [5]:
from scipy import stats
N = 47
q = 1
alpha = 0.05

# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.t.html
qt = stats.t.ppf(q = 1 - alpha/2, df = N-q-2)
t_stat = r*np.sqrt((N-q-2)/1-r**2)
print(f"Kwantyl: {qt:.3f} | Statystyka testowa: {t_stat:.3f}")

Kwantyl: 2.015 | Statystyka testowa: 3.468


Ponieważ $T > q_t$, odrzucamy hipotezę zerową na zadanym poziomie istotności, tzn. mówimy że istnieje niezerowa korelacja cząstkowa $\rho_{12|3}$.

### Zadanie 1b: testowanie korelacji wielokrotnej

Współczynnik korelacji wielokrotnej wyznaczamy ze wzoru $$R_{1|234} = \sqrt{1 - \frac{det C}{C_{11}}},$$
gdzie $C$ to pełna macierz korelacji a $C_{11}$ dopełnienie algebraiczne elementu $1,1$.

In [6]:
det_C = np.linalg.det(corr_full.to_numpy())
C_11 = (-1)**(1+1) * np.linalg.det(corr_full.to_numpy()[1:,1:]) # bez pierwszego wiersza i kolumny
R = np.sqrt(1 - det_C/C_11)
print(f"Wspolczynnik korelacji R_1|234 wyniosl: {R:.3f}")

Wspolczynnik korelacji R_1|234 wyniosl: 0.686


**UWAGA: R nie przyjmuje wartości ujemnych, więc hipoteza jest jednostronna**.
W dalszej części testujemy hipotezę $H_0: R = 0$ przeciwko $H_1: R> 0$ z użyciem statystyki testowej 
$$F = \frac{N-n}{n-1}\frac{\hat R^2}{1 - \hat R^2},$$
gdzie $N$ to liczebność próby, $n$ - wymiarowość rozkładu, a $\hat R$ estymator próbkowy. Statystyka ta ma rozkład F-Snedecora $F(n-1,N-n)$ (patrz wykład 8).

**Uwaga: Hipoteza jest jednostronny. Dlatego nie dzielimy $\alpha$ przez 2. Ale nie ma to nic wspólnego z tym że rozkład jest niesymetryczny!**

In [7]:
n = 4
F_stat = ((N-n)*R**2) / ((n-1)*(1-R**2))
qf = stats.f.ppf(1 - alpha, n-1, N-n)
print(f"Kwantyl: {qf:.3f} | Statystyka testowa: {F_stat:.3f}")

Kwantyl: 2.822 | Statystyka testowa: 12.717


Statystyka testowa leży w obszarze krytycznym (i to bardzo), więc odrzucamy hipotezę zerową i mówimy że istnieje zależność (liniowa?) pierwszej zmiennej od pozostałych.

## Zadanie 2: testowanie hipotez o średniej rozkładu wielowymiarowego
Testujemy hipotezę $\mu = (0,0)$ przeciwko $\mu \neq (0,0)$ z użyciem statystyki testowej
$$\chi^2 = N D^T \Sigma^{-1} D,$$
gdzie $D = \hat X - \mu_0$ jest różnicą między wektorem średnich próbkowych a średnią w hipotezie zerowej. Statystyka ma rozkład z $n$ stopniami swobody (patrz str. 4 wykładu 7).

W tym zadaniu chcemy ustalić jakie wartości powinny znaleźć się na przeciwprzekątnej macierzy $\Sigma$, żeby odrzucić $H_0$ dla $\alpha = 0.05$. Rozkład $\chi^2$ jest jednostronny, więc nie dzielimy wartości przez dwa. Odrzucimy hipotezę dla wartości statystyki testowej większych od kwantyla $q$ na poziomie $1 - \alpha$ rozkładu $\chi^2_n$, który w naszym wypadku wyniósł:

In [8]:
qchi2 = stats.chi2.ppf(1-0.05, 2) # UWAGA! Statystyka chi2 a nie chi!!
qchi2.round(2)

5.99

Wektor $D = (1,0)^T - (0,0)^T = (1,0)^T$, wtedy statystyka testowa wynosi $$N D^T \Sigma^{-1} D = \frac{N}{3 - \tau^2} = \frac{5}{3 - \tau^2}.$$

Mamy $\frac{5}{3 - \tau^2} > q \iff ... \iff |\tau| > \sqrt{3 - \frac{5}{q}}.$ Wyliczamy wartość wyrażenia po prawej stronie:

In [9]:
np.sqrt(3 - 5/qchi2)

1.4715568280775515

**Odp:** Hipotezę należy odrzucić dla wartości $|\tau| > 1.47$.

## Zadanie 3

- S - estymator macierzy kowariancji


### Sposób 1: bez Hotellinga (M.B.)
Możemy przekształcić naszą hipotezę do postaci 1-wymiarowej:
$$2 \mu_1 = mu_2 + \mu_3 \iff (\mu_1 - \mu_2 - $$

## Zadanie 4

## Zadanie 5